## Plotting geospatial data on a map

In this first activity for geoplotlib, you'll combine methodologies learned in the previous exercise and use theoretical knowledge from previous lessons.   
Besides from wrangling data you need to find the area with given attributes.   

Before we can start, however, we need to import our dataset.   
For this activity, we'll work with geo-spatial data that contains all cities with their coordinates and their population.

**Note:**   
This time the dataset is not yet added into the data folder. You have to download it from here:   
https://www.kaggle.com/max-mind/world-cities-database#worldcitiespop.csv

#### Loading the dataset

In [4]:
# importing the necessary dependencies
import numpy as np
import pandas as pd
import geoplotlib

In [5]:
# loading the Dataset (make sure to have the dataset downloaded)
dataset = pd.read_csv('./data/world_cities_pop.csv', dtype={'Region': np.str})

**Note:**   
If we import our dataset without defining the dtype of column *Region* as String, we will get a warning telling out the it has a mixed datatype.   
We can get rid of this warning by explicitly defining the type of the values in this column by using the `dtype` parameter.   
`dtype={'Region': np.str}`

In [6]:
# looking at the data types of each column
dataset.dtypes

Country        object
City           object
AccentCity     object
Region         object
Population    float64
Latitude      float64
Longitude     float64
dtype: object

**Note:**   
Here we can see the dtypes of each column.   
Since the String type is no primitive datatype, it's displayed as `object` here.

In [7]:
# showing the first 5 entries of the dataset
dataset.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,06,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,06,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,06,NaN,42.466667,1.500000
3,ad,aixirvall,Aixirvall,06,NaN,42.466667,1.500000
4,ad,aixovall,Aixovall,06,NaN,42.466667,1.483333


---

#### Mapping `Latitude` and `Longitude` to `lat` and `lon`

Most datasets won't be in the format that you want to have. Some of them might have their latitude and longitude values hidden in a different column.   
This is where the data wrangling skills of lesson 1 are needed.   

For the given dataset, the transformations are easy, we simply need to map the `Latitude` and `Longitude` columns into `lat` and `lon` columns which are used by geoplotlib.

In [8]:
# mapping Latitude to lat and Longitude to lon
dataset['lat'] = dataset['Latitude']
dataset['lon'] = dataset['Longitude']

**Note:**   
Geoplotlibs methods expect dataset columns `lat` and `lon` for plotting. This means your dataframe has to be tranfsormed to resemble this structure.   

---

#### Understanding our data

It's your first day at work, your boss hands you this dataset and wants you to dig into it and find the areas with the most adjacent cities that have a population of more than 100k.   
He needs this information to figure out where to expand next.   

To get a feeling for how many datapoints the dataset contains, we'll plot the whole dataset using dots.

In [9]:
# plotting the whole dataset with dots
geoplotlib.dot(dataset)
geoplotlib.show()

Other than seeing the density of our datapoints, we also need to get some information about how the data is distributed.

In [7]:
# amount of countries and cities
print(len(dataset.groupby(['Country'])), 'Countries')
print(len(dataset), 'Cities')

234 Countries
3173958 Cities


In [8]:
# amount of cities per country (first 20 entries)
dataset.groupby(['Country']).size().head(20)

Country
ad       92
ae      446
af    88749
ag      183
ai       42
al    15123
am     2890
an      269
ao    19560
ar     8738
at    14788
au    10941
aw      115
az    11223
ba    15999
bb      536
bd    26414
be    16218
bf    10468
bg    20106
dtype: int64

In [11]:
# average num of cities per country
dataset.groupby(['Country']).size().agg('mean')

13563.923076923076

Since we are only interested in areas with densely placed cities and high population, we can filter out cities without a population. 

#### Reducing our data

Our dataset has more than 3Mio cities listed. Many of them are really small and can be ignored, given our objective for this activity.   
We only want to look at those cities that have a value given for their population density.

**Note:**   
If you're having trouble filtering your dataset, you can always check back with the activities in lesson1.

In [12]:
# filter for countries with a population entry (Population > 0)
dataset_with_pop = dataset[(dataset['Population'] > 0)]

print('Full dataset:', len(dataset))
print('Cities with population information:', len(dataset_with_pop))

Full dataset: 3173958
Cities with population information: 47980


In [13]:
# displaying the first 5 items from dataset_with_pop
dataset_with_pop.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,lat,lon
6,ad,andorra la vella,Andorra la Vella,07,20430.0,42.500000,1.516667,42.500000,1.516667
20,ad,canillo,Canillo,02,3292.0,42.566667,1.600000,42.566667,1.600000
32,ad,encamp,Encamp,03,11224.0,42.533333,1.583333,42.533333,1.583333
49,ad,la massana,La Massana,04,7211.0,42.550000,1.516667,42.550000,1.516667
53,ad,les escaldes,Les Escaldes,08,15854.0,42.500000,1.533333,42.500000,1.533333


In [14]:
# showing all cities with a defined population with a dot density plot
geoplotlib.dot(dataset_with_pop)
geoplotlib.show()

**Note:**   
Not only the execution time of the visualization has been decreased but we already can see where the areas with more cities are.   

Following the request from our boss, we shall only consider areas that have a high density of adjacent cities with a population of more than 100k.

In [15]:
# dataset with cities with population of >= 100k
dataset_100k = dataset_with_pop[(dataset_with_pop['Population'] >= 100_000)]

print('Cities with a population of 100k or more:', len(dataset_100k))

Cities with a population of 100k or more: 3527


In [16]:
# displaying all cities >= 100k population with a fixed bounding box (WORLD) in a dot density plot
from geoplotlib.utils import BoundingBox

geoplotlib.dot(dataset_100k)
geoplotlib.set_bbox(BoundingBox.WORLD)
geoplotlib.show()

**Note:**   
In order to get the same view on our map every time, we can set the bounding box to the constant viewport declared in the geoplotlib library.   
We can also instantiate the BoundingBox class with values for north, west, south, and east.

---

#### Finding the best area

After reducing our data, we can now use more complex plots to filter down our data even more.   
Thinking back to the first exercise, we've seen that histograms and voronoi plots can give us a quick visual representation of the density of data.

**Note:**   
Try playing around with the different color maps of the plotting methods, sometimes using other colors does not only improve the visuals but also the amount of information you can take from the visualization.

In [17]:
# using filled voronoi to find dense areas
geoplotlib.voronoi(dataset_100k, cmap='hot_r', max_area=1e3, alpha=255)
geoplotlib.show()

In the voronoi plot we can see tendencies.   
Germany, Great Britain, Nigeria, India, Japan, Java, the East Coast of the USA, and Brasil stick out.   
We can now again filter our data and only look at those countries to find the best suited.   

---

#### Final call

After meeting with your boss, he tells you that we want to stick to Europe when it comes to expanding.   
Filter your data for Germany and Great Britain only and decide which area is your final proposal.

In [18]:
# filter 100k dataset for cities in Germany and GB
dataset_europe = dataset_100k[(dataset_100k['Country'] == 'de') | (dataset_100k['Country'] == 'gb')]

print('Cities in Germany or GB with population >= 100k:', len(dataset_europe))

Cities in Germany or GB with population >= 100k: 150


In [19]:
# using Delaunay triangulation to find the most dense aree
geoplotlib.delaunay(dataset_europe, cmap='hot_r')
geoplotlib.show()

Looking at our delaunay visualization, we can quickly see that area around Cologne and Düsseldorf stick out.   
With those insights, we can now get back to our boss and talk about what we found out.

**Note:**   
As mentioned before, it's important to know which visualization type helps you achieve the best insights.   
We e.g. could've simply used a dot density map in the final call which would have also given us an idea about where there are most cities.   
However delaunay triangulation is a good approach here that makes details pop nearly instantly.